In [ ]:
import os
from dotenv import load_dotenv
from langchain.chat_models import  ChatOpenAI

In [ ]:
load_dotenv()

In [ ]:
chat = ChatOpenAI(
    openai_api_key=os.environ.get("OPENAI_API_KEY"),
    model = "gpt-3.5-turbo"
)

In [ ]:
chat

In [ ]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage

In [ ]:
messages = [
    SystemMessage(content="You are a helpful assistant. Your name is Friday"),
    HumanMessage(content= "Hi Friday, how are you today?"),
    SystemMessage(content="Hi, I'm great thank you. How can I help you?"),
    HumanMessage(content="I would like to understand God particle.")
]

In [ ]:
res = chat(messages)
res

In [ ]:
from datasets import load_dataset
import pinecone